In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

In [11]:
train_data = pd.read_csv('titanic_train.csv')
test_data = pd.read_csv('titanic_test.csv')

print(train_data.shape)
print(test_data.shape)

(891, 12)
(418, 11)


In [13]:
train_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [14]:
test_data.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


The survived column, the target variable has been removed from the test set

<ol>
<li> <b>Survived</b>: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.</li>
<li> <b>Pclass</b>: passenger class.</li>
<li> <b>Name, Sex, Age</b>: self-explanatory</li>
<li> <b>SibSp</b>: how many siblings & spouses of the passenger aboard the Titanic.</li>
<li> <b>Parch</b>: how many children & parents of the passenger aboard the Titanic.</li>
<li><b>Ticket</b>: ticket id</li>
<li> <b>Fare:</b> price paid (in pounds)</li>
<li> <b>Cabin:</b> passenger's cabin number</li>
<li> <b>Embarked:</b> where the passenger embarked the Titanic</li>
</ol>

<h3>Checking how much of the data is missing</h3>

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Age**, **Cabin** and **Embarked** have null values. Each of these attributes have **19%**,**77%** and **<1%** have missing values, respectively. Since embarked is missing so many values, I will exclude this attribute from the model. With **age**, I will use the median age to fill in the missing values.

In [16]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Only 38% Survived. That's close enough to 40%, so accuracy will be a reasonable metric to evaluate our model.
* The mean Fare was £32.20, which does not seem so expensive (but it was probably a lot of money back then).
* The mean Age was less than 30 years old.

In [17]:
train_data.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [18]:
train_data.Pclass.value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [19]:
train_data.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [20]:
train_data.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [21]:
train_data.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Embarked is telling us where the passangers embarked from.C=Cherbourg, Q=Queenstown, S=Southampton.

# Building preprocessing pipelines

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [24]:
# Builing a pipeline for numerical attributes
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    
    ("select_numeric", DataFrameSelector(['Age','SibSp','Parch','Fare'])),
    ("imputer", SimpleImputer(strategy="median")),
])

In [25]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ],
       [38.    ,  1.    ,  0.    , 71.2833],
       [26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [28.    ,  1.    ,  2.    , 23.45  ],
       [26.    ,  0.    ,  0.    , 30.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ]])

In [26]:
# need an imputer for the string categorical columns
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [29]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    
    ('select_cat', DataFrameSelector(["Pclass", "Sex", "Embarked"])),
    ('imputer', MostFrequentImputer()),
    ('cat_encoder', OneHotEncoder(sparse=False))
])

In [30]:
cat_pipeline.fit_transform(train_data)

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [31]:
#joining numerical and categorical pipelines

from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])

Now there is a preprocessing pipeline that takes raw data and outputs numerical input features that we can feed into the machine learning model

In [34]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train.shape

(891, 12)

In [37]:
y_train = train_data.Survived
y_train.shape

(891,)

# Training Classifiers
<h2>SVC</h2>

In [39]:
from sklearn.svm import SVC
svm_clf = SVC(gamma='auto')
svm_clf.fit(X_train, y_train)

SVC(gamma='auto')

In [40]:
# model is trained and now we will make predictions
X_test = preprocess_pipeline.fit_transform(test_data)
y_pred = svm_clf.predict(X_test)

In [43]:
#Using cross-validation to get an idea of how good a model is

from sklearn.model_selection import cross_val_score
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()


0.7329588014981274

This means there is 73% accuracy to this model is accuractely predicting survival. Let us find a better model. I am going to try **RandomForestClassifier**

In [46]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.8126466916354558

The **RandomForestClassifier** has a higher accuracy of 81%. This means this model accurately predicts 8% better than the SVC model